In [1]:
import os
print(os.getcwd())
os.chdir('/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1')
print(os.getcwd())

/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1/tests
/Users/boyuren/Documents/multi_head_graph_rag/MH-GRAG-V1


In [2]:
import pandas as pd

origin_multihop_rag=pd.read_csv("data/benchmark_datasets/multihop_rag/corpus/corpus_chunks.csv")
final_entities=pd.read_csv("export/multihop_rag/multihop_rag_final_entities.csv")
final_relationships=pd.read_csv("export/multihop_rag/multihop_rag_final_relationships_v1.csv")

# Multihop-RAG 原始数据

In [4]:
origin_multihop_rag.head(10)

,id,category,author,published_at,title,source,chunk,token_count
0,0,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"table of contents table of contents echo , fir...",300
1,1,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,amazon ' s invite - only deals for prime membe...,304
2,2,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"99 ) , our new favorite ipad deal at amazon is...",317
3,3,entertainment,NaN,2023-11-27 08:45:59+00:00,200+ of the best deals from Amazon's Cyber Mon...,Mashable,"on the outside , the go 3 is definitely giving...",386
4,4,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,"etf provider betashares , which manages $ 30 b...",314
5,5,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,4 per cent . the lowdown novus capital senior ...,309
6,6,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,55 per cent from 4 . 54 per cent late on monda...,304
7,7,business,Stan Choe,2023-09-26 19:11:30+00:00,ASX set to drop as Wall Street’s September slu...,The Sydney Morning Herald,"on wall street , the vast majority of stocks f...",307
8,8,business,NaN,2023-10-06 21:31:00+00:00,Amazon sellers sound off on the FTC's 'long-ov...,Cnbc | World Business News Leader,a worker sorts out parcels in the outbound doc...,323
9,9,business,NaN,2023-10-06 21:31:00+00:00,Amazon sellers sound off on the FTC's 'long-ov...,Cnbc | World Business News Leader,he opted for the latter . fred ruckel ' s comp...,308


# Multihop-RAG Extracted 数据展示

In [4]:
# 设置pandas显示选项，增加最大列宽度
pd.set_option('display.max_colwidth', 200)  # 将200替换为你想要的长度
# 筛选 entity_name 长度小于30且 description 长度大于150 的行
filtered_df = final_entities[
    (final_entities['entity_name'].str.len() < 30) & 
    (final_entities['entity_name'].str.len() > 10) & 
    (final_entities['description'].str.len() > 50)
]

# 查看筛选后的数据框
filtered_df.head(10)


,entity_name,entity_type,description,id
3,Bolt Mobility,COMPANY,Bolt Mobility is introducing distance-based pricing,4
4,Crimson Tide,TEAM,Crimson Tide is the nickname of the Alabama football team,5
6,Lamar Jackson,PERSON,Lamar Jackson is a quarterback for the Baltimore Ravens.,7
8,Quarter - Final,CONCEPT,The quarter - final is the stage of a tournament where the four winning teams from the group stage compete to advance to the semi - final,9
15,audible audiobooks,PRODUCT,The Kindle Oasis supports audible audiobooks via bluetooth.,16
22,cranberries,PRODUCT,Marcus Urann introduced a way to put cranberries in a can in 1912. Cranberries are one of the few fruits native to the northern United States.,23
26,ezekiel elliott,PLAYER,Ezekiel Elliott is an American football running back for the Dallas Cowboys of the National Football League.,27
28,folklore and evermore albums,ALBUM,folklore and evermore albums are the tenth and eleventh studio albums by Taylor Swift.,29
31,gm meetings,EVENT,The GM meetings are an annual gathering of general managers from Major League Baseball teams.,32
36,hiv - positive,CONCEPT,hiv - positive refers to people who have been diagnosed with HIV.,37


In [5]:
# 设置pandas显示选项，增加最大列宽度
pd.set_option('display.max_colwidth', 200)  # 将200替换为你想要的长度
# 筛选 entity_name 长度小于30且 description 长度大于150 的行
filtered_df = final_relationships[
    (final_relationships['source_entity'].str.len() < 30) & 
    (final_relationships['source_entity'].str.len() > 5) & 
    (final_relationships['target_entity'].str.len() < 30) & 
    (final_relationships['target_entity'].str.len() > 5) & 
    (final_relationships['relationship_description'].str.len() > 50)
]

# 查看筛选后的数据框
filtered_df.head(10)

,source_entity,target_entity,relationship_type,relationship_description,source_id,target_id
12,Epoch Times,Falun Gong,Part-of,The Epoch Times is part of the Falun Gong organization.,3835,3963
16,bowser,flower kingdom,Located-in,bowser's mecha fortress is located in the flower kingdom,11848,14341
27,bowser,flower kingdom,Part-of,bowser's mecha fortress is part of the flower kingdom,11848,14341
50,Super Mario Bros. Wonder,Nintendo,Produced-by,Super Mario Bros. Wonder was developed by Nintendo.,8998,7038
52,Super Mario Bros. Wonder,Super Mario World,Compared-to,Super Mario Bros. Wonder is often compared to Super Mario World.,8998,9000
53,Berlin Wall,Gaza Strip,Located-in,The Berlin Wall was located in between the Gaza Strip and Israel.,2091,4285
54,Berlin Wall,Israel,Located-in,The Berlin Wall was located in between the Israel and the Gaza Strip.,2091,4990
55,Berlin Wall,Gaza Strip,Located-in,The Berlin Wall was located in between the Israel and the Gaza Strip.,2091,4285
56,Gaza Strip,Israel,Located-in,The Gaza Strip is located in between the Israel and the Berlin Wall.,4285,4990
58,barrier,israel,Located-in,"The sand clock had only proved itself in stopping illegal immigrants on the Israel-Egypt border, and the same fence that then-US president Donald Trump enthused over, and wanted to duplicate on th...",11494,15816


# 加载Embedding数据

In [4]:
from src.utils.embedding_utils import load_df_from_csv_with_embedding
from src.utils.config import OUTPUT_PATH

final_entities=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_entities_with_embeddings.csv",['entity_embedding'])
final_entities

,entity_name,entity_type,description,id,entity_embedding
0,49ers,ORGANIZATION,49ers is a football team,1,"[0.04144171625375748, -0.007414397317916155, -..."
1,Barbarians,CONCEPT,a character class in the game,2,"[0.0122455395758152, -0.007463064510375261, -0..."
2,Bills,ORGANIZATION,The team that Allen plays for,3,"[0.030726933851838112, 0.0030639127362519503, ..."
3,Bolt Mobility,COMPANY,Bolt Mobility is introducing distance-based pr...,4,"[0.004269156605005264, -0.011844240128993988, ..."
4,Crimson Tide,TEAM,Crimson Tide is the nickname of the Alabama fo...,5,"[0.004862545058131218, -0.003534935647621751, ..."
...,...,...,...,...,...
23565,家庭自动化,CONCEPT,家庭自动化是指通过网络连接和控制家庭设备的技术。,23569,"[0.02464906871318817, 0.004047214984893799, -0..."
23566,昂贵的,CONCEPT,昂贵的是形容 soundbar 的形容词,23570,"[0.04650348424911499, 0.02652094140648842, -0...."
23567,智能扬声器,CONCEPT,智能扬声器是一种可以通过网络连接和控制的扬声器。,23571,"[0.03134648874402046, 0.013866862282156944, -0..."
23568,杜比 atmos,CONCEPT,杜比 atmos is a surround sound technology.,23572,"[0.01925036683678627, 0.018349502235651016, -0..."


In [5]:
final_relationships=load_df_from_csv_with_embedding(f"{OUTPUT_PATH}/multihop_rag/final_relationships_with_embeddings.csv",['relationship_embedding'])
final_relationships

,source_entity,target_entity,relationship_type,relationship_description,source_id,target_id,relationship_embedding
0,Li,Falun Gong,Affiliated-with,Li is affiliated with Falun Gong.,5841,3963,"[0.006164404563605785, 0.004094846546649933, -..."
1,Epoch Times,Li,Worked-for,Epoch Times worked for Li.,3835,5841,"[0.014970953576266766, -0.01761895790696144, 0..."
2,The Epoch Times,Location,Located-in,The Epoch Times is located in multiple countries.,9282,5932,"[0.022465074434876442, -0.007946091704070568, ..."
3,Microsoft,The Epoch Times,Affiliated-with,Microsoft is affiliated with The Epoch Times.,6510,9282,"[0.024022087454795837, -0.024629345163702965, ..."
4,California,The Epoch Times,Located-in,California is located in The Epoch Times.,2502,9282,"[0.02883162721991539, -0.014359557069838047, -..."
...,...,...,...,...,...,...,...
26033,Lille,Alphonso Davies,Part-of,Lille is a part of Davies,5873,1515,"[0.009421243332326412, 0.025672562420368195, -..."
26034,Canada Soccer,"November 18, 2023",Happened-at,"November 18, 2023 is an event that happened at...",2535,7125,"[0.05737772211432457, 0.003261846723034978, -0..."
26035,David,Lille,Part-of,David is a part of Lille,3263,5873,"[-0.006869114935398102, 0.004632634576410055, ..."
26036,David,Milan Borjan,Part-of,David is a part of Borjan,3263,6561,"[0.024512803182005882, -0.0016717436956241727,..."


# Leiden/GNN聚类评估

In [7]:
from src.gnn_clustering.data_loader import load_custom_data
from src.gnn_clustering.utils import get_device
from tests.utils import test_model_performance
import numpy as np

# 设置设备
device = get_device()

# 填充空的embedding
expected_size = 1024
final_entities['entity_embedding'] = final_entities['entity_embedding'].apply(
    lambda x: np.zeros(expected_size) if x is None or len(x) == 0 else x
)
final_relationships['relationship_embedding'] = final_relationships['relationship_embedding'].apply(
    lambda x: np.zeros(expected_size) if x is None or len(x) == 0 else x
)
# 去除异常边
entity_ids = final_entities['id'].unique()
final_relationships = final_relationships[
    final_relationships['source_id'].isin(entity_ids) & final_relationships['target_id'].isin(entity_ids)
]


# 数据加载
data = load_custom_data(final_entities, final_relationships)
data = data.to(device)

# 执行测试
report_df = test_model_performance(data=data, device=device, num_tests=3)

In [ ]:
report_df